## BARREL Create Background Model


### Setup
Start by importing libraries and loading data from a potentially interesting event.

In this guide, we are going to use interactive plots, so `%matplotlib ipympl` should be set.

In [ ]:
%matplotlib ipympl
import pyspedas, pytplot, pprint, numpy


For our example, we will look at data form flight 1G from January 17th - 19th, 2013.

FSPC and SSPC data can be downloaded with the `pyspedas.barrel` helper functions:

In [ ]:
trange=['2013-01-17', '2013-01-19']

pyspedas.barrel.fspc(
    trange=trange,
    probe='1g'
)

pyspedas.barrel.sspc(
    trange=trange,
    probe='1g'
)

### Background subtraction
Plot FSPC1 for the loaded data and visually determine the start and stop locations for the background selection.

In [ ]:
pytplot.tplot('brl1G_SSPC')
pytplot.tplot('brl1G_FSPC1')

By moving the mouse cursor over a quiet area of the plot, we can estimate values for the start and stop times.
In this way we can find one or more periods of time to use for background calculation.

These start and stop times can be stored in a list of tuples:

In [ ]:
background_periods=[("2013-01-17/17:25", "2013-01-17/20:35"), ("2013-01-18/09:35", "2013-01-18/12:04")]

Similarly, we can use the plot to estimate the time period of the event that we are interested in:

In [ ]:
event_period=("2013-01-17/01:54", "2013-01-17/03:24")

In [ ]:
ts, cnts, energy_levels = pytplot.get_data('brl1G_SSPC')
spec = pyspedas.barrel.average_event_spectrum(ts, cnts, energy_levels, background_periods, event_period)
pytplot.store_data("brl1G_Event_Spec", data={'y':energy_levels, 'x':spec})
#pytplot.options("brl1G_Event_Spec", "xlog_slice", True)
pytplot.options("brl1G_Event_Spec", "ylog", True)
pytplot.tplot("brl1G_Event_Spec")

Next, we will extract the SSPC data during that time range so we can calcuate the background.

In [ ]:
#use pytplot to extract the sspc data
ts, cnts, energy_levels = pytplot.get_data('brl1G_SSPC')

#use the background subtraction function
bkg_sub_cnts = pyspedas.barrel.bg_sub(ts, cnts, background_periods)
#pytplot.store_data('brl1G_SSPC_BGSUB', data={'x':ts, 'y': bkg_sub_cnts})

#Remove any previous version of this variable, then make a copy of the unsubtracted variable and replace its data with that of the subrated spectrum
pytplot.del_data('brl1G_SSPC_BGSUB')
pytplot.tplot_copy('brl1G_SSPC', 'brl1G_SSPC_BGSUB')
print(pytplot.data_quants['brl1G_SSPC_BGSUB'].values.shape)
print(pytplot.data_quants['brl1G_SSPC'].values.shape)

#pytplot.tplot('brl1G_SSPC_BGSUB')
#import pprint
#pprint.pprint(pytplot.data_quants['brl1G_SSPC_BGSUB'])
#pytplot.tplot('brl1G_SSPC_BGSUB')


In [ ]:
from copy import deepcopy
import pprint
import json
old_name = 'brl1G_SSPC'
new_name = 'brl1G_SSPC_BGSUB_1'
attr_dict = deepcopy(pytplot.data_quants[old_name].attrs)
#pprint.pprint(attr_dict)
data_dict = {}
data_dict['x'] = pytplot.data_quants[old_name].coords['time'].values

data_dict['y'] = pytplot.data_quants[old_name].values
if len(data_dict['y'].shape) <= 1:
    print("balls")
    pass
else:
    for c in pytplot.data_quants[old_name].coords:
        if c == 'time':
            continue
        data_dict[c] = pytplot.data_quants[old_name].coords[c].values
print(json.dump(data_dict))
#pytplot.store_data(new_name, data=data_dict)
#pytplot.data_quants[new_name].attrs = attr_dict
#pytplot.tplot(new_name)

Finally, we can create a new tplot variable containing out background-subtracted spectra and plot it.

In [ ]:
pytplot.store_data("brl1G_SSPC_SUB_BKG1", data={'x':ts, 'y': bkg_sub_cnts, 'v': energy_levels})
print(pytplot.tplot_names())
pytplot.tplot("brl1G_SSPC_SUB_BKG1")

In [ ]:
import pprint
#pprint.pprint(pytplot.data_quants['brl1G_SSPC'])
pytplot.del_data('brl1G_SSPC_BGSUB')
pytplot.tplot_copy('brl1G_SSPC', 'brl1G_SSPC_BGSUB')
pytplot.tplot('brl1G_SSPC_BGSUB')